### (실패 한 것에 대한 정리)
- 정규화 처리 과정 변경 -> 기존에 시도하려 했지만 차원 수도 늘어나고 복잡해지면서 문제가 생기는 듯함.
- 주기적 데이터(Month, Day): sin, cos 변환으로 원형 관계를 반영.
- 범주형 데이터
- 고유값이 적은 변수 → 원-핫 인코딩.
- 고유값이 많은 변수 → 라벨 인코딩 또는 Embedding.
이렇게 해보려고 하였으나 차원 수가 커지고 복잡해지는 문제가 생겨서 날짜 데이터를 수치적으로 가져가기 위해서 0~1로 정규화 시켜서 적용하였음. 


# 정규화 처리 과정 변경된 것
-  date_columns = ['Year', 'Month', 'Day', 'Birth Year', 'Birth Month']-> 날짜 데이터도 0~1로 표현되게 정규화 처리함.
- 연속형 데이터: Min-Max Scaling으로 0~1로 정규화. -> ReLU는 음수 값을 0으로 바꿔버리기 때문에, StandardScaler에서 생성된 음수 값이 문제를 일으킬 가능성이 있을 수 있어서

In [56]:
# 기본 Import
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import torch
import torch.nn as nn
from sklearn.manifold import TSNE

# 학습에 사용되는 자잘한 것들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler


In [ ]:
class BaseModel(nn.Module):
    """
    모델 구조 수정 금지.
    """
    def __init__(self, encoding_dim, cat_features, num_features, num_classes, cat_cardinalities):
        super(BaseModel, self).__init__()
        # cat_cardinalities는 각 범주형 변수의 고유값 개수 리스트
        self.cat_embeddings = nn.ModuleList([nn.Embedding(cardinality, 5) for cardinality in cat_cardinalities])
        self.fc_cat = nn.Linear(len(cat_features) * 5 + len(num_features), 64)
        self.encoder = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x_cat, x_num):
        # Apply embedding layers
        embeddings = [emb(x_cat[:, i]) for i, emb in enumerate(self.cat_embeddings)]
        #print('len(embeddings : )',len(embeddings))
        #print('len(x_num) : ',len(x_num))
        x = torch.cat(embeddings + [x_num], dim=1)
        #print('len(x) : ',len(x))
        x = self.fc_cat(x)
        encoded = self.encoder(x)
        out = self.classifier(encoded)
        # print(out)
        return out


In [ ]:
def encode_and_standardize_data(data, mode):
    label_encoders = {}
    cat_cardinalities = []

    # 기존 범주형 열 정의
    categorical_columns_train = ['Card Brand', 'Card Type', 'Card Number', 'Expires', 'Acct Open Date', 'Is Fraud?', 'Error Message']
    categorical_columns_test = ['Card Brand', 'Card Type', 'Card Number', 'Expires', 'Acct Open Date', 'Error Message']
    data['Error Message'] = data['Error Message'].fillna('None')
    categorical_columns = categorical_columns_train if mode == 'Train' else categorical_columns_test

    # 범주형 데이터 레이블 인코딩
    for col in categorical_columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le
        cat_cardinalities.append(data[col].nunique())

    # Zipcode와 Merchandise Code 처리 (레이블 인코딩)
    for col in ['Zipcode', 'Merchandise Code']:
        data[col] = (data[col] // 100).astype(int)
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col])
        label_encoders[col] = le
        cat_cardinalities.append(data[col].nunique())

    # Boolean 열 처리 (Has Chip)
    data['Has Chip'] = np.where(data['Has Chip'] == True, 1, 0)
    cat_cardinalities.append(data['Has Chip'].nunique())
    
    # 날짜 데이터 정규화
    date_columns = ['Year', 'Month', 'Day', 'Birth Year', 'Birth Month']
    for col in date_columns:
        if data[col].max() != data[col].min():  # 값이 서로 다를 때만 정규화 수행
            data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
        else:  # 값이 동일하면 0으로 설정
            data[col] = 0

    # 연속형 데이터 정규화
    continuous_columns = [
        'Current Age', 'Retirement Age', 'Per Capita Income - Zipcode',
        'Yearly Income', 'Total Debt', 'Credit Score', 'Credit Limit', 'Amount'
    ]
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()  # MinMaxScaler로 변경
    data[continuous_columns] = scaler.fit_transform(data[continuous_columns])

    # 범주형 및 수치형 열 분리
    categorical_columns += ['Zipcode', 'Merchandise Code', 'Has Chip']
    cat_features = data[categorical_columns].astype(int)  # Ensure categorical features are integer
    num_features = data[continuous_columns + date_columns]  # 날짜와 연속형 데이터를 결합

    return cat_features, num_features, cat_cardinalities


In [ ]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

x_cat_train, x_num_train, cat_cardinalities_train = encode_and_standardize_data(train_data, mode='Train')
x_cat_test, x_num_test, cat_cardinalities_test = encode_and_standardize_data(test_data, mode='Test')

ValueError: y contains previously unseen labels: 2

In [ ]:
print(x_cat_test.iloc[1], x_num_test.iloc[1], cat_cardinalities_test,sep='\n')

Card Brand             2
Card Type              1
Card Number         3088
Expires               57
Acct Open Date       203
Error Message         20
Zipcode               52
Merchandise Code      24
Has Chip               1
Name: 1, dtype: int64
Current Age                    0.414634
Retirement Age                 0.551724
Per Capita Income - Zipcode    0.179460
Yearly Income                  0.329990
Total Debt                     0.286566
Credit Score                   0.574924
Credit Limit                   0.171828
Amount                         0.100280
Year                           0.000000
Month                          0.000000
Day                            0.166667
Birth Year                     0.585366
Birth Month                    0.909091
Name: 1, dtype: float64
[3, 3, 3850, 60, 297, 22, 555, 37, 2]


In [ ]:
# Torch tensor로 변환
x_cat_train_tensor = torch.tensor(x_cat_train.values, dtype=torch.long)  # 정수형
x_num_train_tensor = torch.tensor(x_num_train.values, dtype=torch.float32)  # 실수형

x_cat_test_tensor = torch.tensor(x_cat_test.values, dtype=torch.long)
x_num_test_tensor = torch.tensor(x_num_test.values, dtype=torch.float32)


In [ ]:
num_classes = 2  # 예: Is Fraud? 이진 분류
encoding_dim = 64 # 이게 이제 

model = BaseModel(encoding_dim=encoding_dim, cat_features=x_cat_train.columns, num_features=x_num_train.columns, num_classes=num_classes, cat_cardinalities=cat_cardinalities_train)

# 모델 출력 테스트
output = model(x_cat_train_tensor, x_num_train_tensor)
print(output)

tensor([[ 0.2268, -0.2595],
        [ 0.2205, -0.2598],
        [ 0.2246, -0.2628],
        ...,
        [ 0.2278, -0.2448],
        [ 0.2338, -0.2504],
        [ 0.2336, -0.2514]], grad_fn=<AddmmBackward0>)


In [ ]:
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Train-Test Split
x_cat_train_split, x_cat_val_split, x_num_train_split, x_num_val_split = train_test_split(
    x_cat_train_tensor, x_num_train_tensor, test_size=0.2, random_state=42
)

# Dataset 및 DataLoader 정의
train_dataset = torch.utils.data.TensorDataset(x_cat_train_split, x_num_train_split)
val_dataset = torch.utils.data.TensorDataset(x_cat_val_split, x_num_val_split)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

# Loss Function, Optimizer 정의
criterion = nn.CrossEntropyLoss()  # 다중 클래스 분류의 경우
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train Loop 구현
def train_model(model, train_loader, val_loader, num_epochs=10):
    model.train()  # 모델을 학습 모드로 설정
    for epoch in range(num_epochs):
        total_loss = 0
        for x_cat_batch, x_num_batch in train_loader:
            optimizer.zero_grad()  # 기울기 초기화
            outputs = model(x_cat_batch, x_num_batch)
            loss = criterion(outputs, torch.randint(0, 2, (x_cat_batch.size(0),)))  # 임시 타겟 (예: binary class)
            loss.backward()  # 역전파
            optimizer.step()  # 가중치 업데이트
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

        # Validation Loop
        model.eval()  # 모델을 평가 모드로 설정
        val_loss = 0
        with torch.no_grad():
            for x_cat_batch, x_num_batch in val_loader:
                outputs = model(x_cat_batch, x_num_batch)
                loss = criterion(outputs, torch.randint(0, 2, (x_cat_batch.size(0),)))  # 임시 타겟
                val_loss += loss.item()
        print(f"Validation Loss: {val_loss:.4f}")
        model.train()  # 다시 학습 모드로 전환


Epoch 1/10, Loss: 14252.4794
Validation Loss: 3563.1953
Epoch 2/10, Loss: 14252.3776
Validation Loss: 3563.2070
Epoch 3/10, Loss: 14252.5173
Validation Loss: 3563.0763
Epoch 4/10, Loss: 14252.3241
Validation Loss: 3563.2234
Epoch 5/10, Loss: 14252.1432
Validation Loss: 3563.1689
Epoch 6/10, Loss: 14252.3973
Validation Loss: 3563.0662
Epoch 7/10, Loss: 14252.4731
Validation Loss: 3563.2320
Epoch 8/10, Loss: 14252.0650
Validation Loss: 3562.9845
Epoch 9/10, Loss: 14252.4512
Validation Loss: 3563.1127
Epoch 10/10, Loss: 14252.5042
Validation Loss: 3563.0195


IndexError: index out of range in self